In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train_data.drop(['PassengerId','Name','Ticket','Cabin'], axis =1, inplace = True)
test_data.drop(['Name','Ticket','Cabin'], axis =1, inplace = True)
train_data['Age'].fillna(train_data['Age'].median(), inplace = True)
test_data['Age'].fillna(train_data['Age'].median(), inplace = True)
y = train_data.Survived
Id = test_data.PassengerId
train_data.drop(['Survived'], axis =1, inplace = True)
test_data.drop(['PassengerId'], axis =1, inplace = True)

In [5]:
train_data.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [6]:
train_data= pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)
train_data_f,test_data_f = train_data.align(test_data, join = 'inner', axis = 1)

In [7]:
train_data_f.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1


In [8]:
test_data_f.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1


In [9]:
train_data_f.isna().any()

Pclass        False
Age           False
SibSp         False
Parch         False
Fare          False
Sex_female    False
Sex_male      False
Embarked_C    False
Embarked_Q    False
Embarked_S    False
dtype: bool

In [10]:
test_data_f.isna().any()

Pclass        False
Age           False
SibSp         False
Parch         False
Fare           True
Sex_female    False
Sex_male      False
Embarked_C    False
Embarked_Q    False
Embarked_S    False
dtype: bool

In [11]:
test_data_f['Fare'].fillna(test_data_f['Fare'].mean(), inplace = True)

In [12]:
from sklearn.ensemble import RandomForestRegressor
obj = RandomForestRegressor(random_state = 0)
obj.fit(train_data_f,y)
prediction = obj.predict(test_data_f)
prediction

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


array([0.11      , 0.21      , 0.59      , 0.62      , 0.58      ,
       0.0425    , 0.128     , 0.08      , 0.95      , 0.02      ,
       0.        , 0.14216667, 1.        , 0.28      , 0.99      ,
       0.975     , 0.04164286, 0.71      , 0.69      , 0.238     ,
       0.53      , 0.81      , 1.        , 0.48      , 0.91      ,
       0.01      , 1.        , 0.66      , 0.67333333, 0.15      ,
       0.01      , 0.02      , 0.79      , 0.15      , 0.61625   ,
       0.49      , 0.03      , 0.08      , 0.01      , 0.47014286,
       0.312     , 0.60383333, 0.095     , 1.        , 0.99      ,
       0.014     , 0.24666667, 0.11514495, 0.98      , 0.73      ,
       0.55      , 0.19      , 0.92      , 0.86      , 0.37      ,
       0.05      , 0.01      , 0.        , 0.        , 1.        ,
       0.06666667, 0.3       , 0.        , 0.89      , 0.68      ,
       0.89      , 0.86      , 0.04      , 0.1325    , 0.91      ,
       0.67816919, 0.02      , 0.28      , 0.0625    , 1.     

In [13]:
prediction = prediction>=0.5
prediction = pd.Series(prediction.astype(int))
prediction

0      0
1      0
2      1
3      1
4      1
      ..
413    0
414    1
415    0
416    0
417    1
Length: 418, dtype: int64

In [14]:
columns = {'PassengerId': Id, 'Survived': prediction}
submission = pd.concat(columns, axis =1)
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [15]:
submission.to_csv('submission.csv', index = False)